# FAISS Embeddings Training for : multi-qa-MiniLM-L6-cos-v1
### Import Dependancies

In [1]:
import os
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import pickle

In [2]:
CSV_PATH = "./../cleaned_database/cleaned_final_dataset3.csv"
OUTPUT_DIR = "./../faiss_embeddings1"
BATCH_SIZE = 1000 

In [3]:
def load_data(csv_path):
    df = pd.read_csv(csv_path)
    docs = df["docs"].fillna("").tolist()
    ids = df["id"].tolist()
    metadata = df.drop(columns=["docs"])
    return docs, ids, metadata

In [4]:
def load_model():
    return SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1') 


In [5]:
def create_index(model):
    dim = model.get_sentence_embedding_dimension()
    return faiss.IndexFlatIP(dim)

### Processes the batches

In [6]:
def process_batches(docs, ids, model, index, batch_size):
    all_ids = []
    for start in range(0, len(docs), batch_size):
        end = start + batch_size
        batch_docs = docs[start:end]
        batch_ids = ids[start:end]

        print(f"Processing batch {start // batch_size + 1} | Size: {len(batch_docs)}")
        embeddings = model.encode(batch_docs, normalize_embeddings=True).astype("float32")
        index.add(embeddings)
        all_ids.extend(batch_ids)
    return all_ids


In [7]:
def save_outputs(index, ids, metadata, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    faiss.write_index(index, os.path.join(output_dir, "movie_index.faiss"))
    with open(os.path.join(output_dir, "movie_ids.pkl"), "wb") as f:
        pickle.dump(ids, f)

    metadata.to_csv(os.path.join(output_dir, "movie_metadata.csv"), index=False)

In [8]:
def run_embedding_pipeline():
    docs, ids, metadata = load_data(CSV_PATH)
    model = load_model()
    index = create_index(model)
    final_ids = process_batches(docs, ids, model, index, BATCH_SIZE)
    save_outputs(index, final_ids, metadata, OUTPUT_DIR)

In [ ]:
run_embedding_pipeline()
